<a href="https://colab.research.google.com/github/piyushpradhan22/MLChallenge/blob/master/NBME_E5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
f = open("kaggle.json", 'w')
f.write("{\"username\":\"piyushpradhan22\",\"key\":\"59c6ae034f7e7db2f2740d65be499b09\"}")
f.close()
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [2]:
#"""
!kaggle competitions download -c nbme-score-clinical-patient-notes
!unzip -o /content/nbme-score-clinical-patient-notes.zip
#"""

nbme-score-clinical-patient-notes.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/nbme-score-clinical-patient-notes.zip
  inflating: features.csv            
  inflating: patient_notes.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
%%capture
!pip install transformers

In [4]:
path=''

In [5]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [6]:
df1 = pd.read_csv(path+"train.csv")
df2 = pd.read_csv(path+"patient_notes.csv")
df3 = pd.read_csv(path+"features.csv")
df4 = pd.read_csv(path+"test.csv")
df_test = pd.merge(df4, df2, how='left', on = 'pn_num').merge(df3, how = 'left', on = 'feature_num')
df = pd.merge(df1, df2, how='left', on = 'pn_num').merge(df3, how = 'left', on = 'feature_num') ; del df1, df2, df3,df4
df.pn_history = df.pn_history.str.lower()
df.annotation = df.annotation.str.lower()
df.drop(columns=['case_num_x','case_num_y'], inplace=True)

In [7]:
# incorrect annotation
df.loc[338, 'annotation'] = '["father heart attack"]'
df.loc[338, 'location'] = '["764 783"]'

df.loc[621, 'annotation'] = '["for the last 2-3 months"]'
df.loc[621, 'location'] = '["77 100"]'

df.loc[655, 'annotation'] = '["no heat intolerance","no cold intolerance"]'
df.loc[655, 'location'] = '["285 292;301 312","285 287;296 312"]'

df.loc[1262, 'annotation'] = '["mother thyroid problem"]'
df.loc[1262, 'location'] = '["551 557;565 580"]'

df.loc[1265, 'annotation'] = '[\'felt like he was going to "pass out"\']'
df.loc[1265, 'location'] = '["131 135;181 212"]'

df.loc[1396, 'annotation'] = '["stool , with no blood"]'
df.loc[1396, 'location'] = '["259 280"]'

df.loc[1591, 'annotation'] = '["diarrhoe non blooody"]'
df.loc[1591, 'location'] = '["176 184;201 212"]'

df.loc[1615, 'annotation'] = '["diarrhea for last 2-3 days"]'
df.loc[1615, 'location'] = '["249 257;271 288"]'

df.loc[1664, 'annotation'] = '["no vaginal discharge"]'
df.loc[1664, 'location'] = '["822 824;907 924"]'

df.loc[1714, 'annotation'] = '["started about 8-10 hours ago"]'
df.loc[1714, 'location'] = '["101 129"]'

df.loc[1929, 'annotation'] = '["no blood in the stool"]'
df.loc[1929, 'location'] = '["531 539;549 561"]'

df.loc[2134, 'annotation'] = '["last sexually active 9 months ago"]'
df.loc[2134, 'location'] = '["540 560;581 593"]'

df.loc[2191, 'annotation'] = '["right lower quadrant pain"]'
df.loc[2191, 'location'] = '["32 57"]'

df.loc[2553, 'annotation'] = '["diarrhoea no blood"]'
df.loc[2553, 'location'] = '["308 317;376 384"]'

df.loc[3124, 'annotation'] = '["sweating"]'
df.loc[3124, 'location'] = '["549 557"]'

df.loc[3858, 'annotation'] = '["previously as regular","previously eveyr 28-29 days","previously lasting 5 days","previously regular flow"]'
df.loc[3858, 'location'] = '["102 123","102 112;125 141","102 112;143 157","102 112;159 171"]'

df.loc[4373, 'annotation'] = '["for 2 months"]'
df.loc[4373, 'location'] = '["33 45"]'

df.loc[4763, 'annotation'] = '["35 year old"]'
df.loc[4763, 'location'] = '["5 16"]'

df.loc[4782, 'annotation'] = '["darker brown stools"]'
df.loc[4782, 'location'] = '["175 194"]'

df.loc[4908, 'annotation'] = '["uncle with peptic ulcer"]'
df.loc[4908, 'location'] = '["700 723"]'

df.loc[6016, 'annotation'] = '["difficulty falling asleep"]'
df.loc[6016, 'location'] = '["225 250"]'

df.loc[6192, 'annotation'] = '["helps to take care of aging mother and in-laws"]'
df.loc[6192, 'location'] = '["197 218;236 260"]'

df.loc[6380, 'annotation'] = '["No hair changes","No skin changes","No GI changes","No palpitations","No excessive sweating"]'
df.loc[6380, 'location'] = '["480 482;507 519","480 482;499 503;512 519","480 482;521 531","480 482;533 545","480 482;564 582"]'

df.loc[6562, 'annotation'] = '["stressed due to taking care of her mother","stressed due to taking care of husbands parents"]'
df.loc[6562, 'location'] = '["290 320;327 337","290 320;342 358"]'

df.loc[6862, 'annotation'] = '["stressor taking care of many sick family members"]'
df.loc[6862, 'location'] = '["288 296;324 363"]'

df.loc[7022, 'annotation'] = '["heart started racing and felt numbness for the 1st time in her finger tips"]'
df.loc[7022, 'location'] = '["108 182"]'

df.loc[7422, 'annotation'] = '["first started 5 yrs"]'
df.loc[7422, 'location'] = '["102 121"]'

df.loc[8876, 'annotation'] = '["No shortness of breath"]'
df.loc[8876, 'location'] = '["481 483;533 552"]'

df.loc[9027, 'annotation'] = '["recent URI","nasal stuffines, rhinorrhea, for 3-4 days"]'
df.loc[9027, 'location'] = '["92 102","123 164"]'

df.loc[9938, 'annotation'] = '["irregularity with her cycles","heavier bleeding","changes her pad every couple hours"]'
df.loc[9938, 'location'] = '["89 117","122 138","368 402"]'

df.loc[9973, 'annotation'] = '["gaining 10-15 lbs"]'
df.loc[9973, 'location'] = '["344 361"]'

df.loc[10513, 'annotation'] = '["weight gain","gain of 10-16lbs"]'
df.loc[10513, 'location'] = '["600 611","607 623"]'

df.loc[11551, 'annotation'] = '["seeing her son knows are not real"]'
df.loc[11551, 'location'] = '["386 400;443 461"]'

df.loc[11677, 'annotation'] = '["saw him once in the kitchen after he died"]'
df.loc[11677, 'location'] = '["160 201"]'

df.loc[12124, 'annotation'] = '["tried Ambien but it didnt work"]'
df.loc[12124, 'location'] = '["325 337;349 366"]'

df.loc[12279, 'annotation'] = '["heard what she described as a party later than evening these things did not actually happen"]'
df.loc[12279, 'location'] = '["405 459;488 524"]'

df.loc[12289, 'annotation'] = '["experienced seeing her son at the kitchen table these things did not actually happen"]'
df.loc[12289, 'location'] = '["353 400;488 524"]'

df.loc[13238, 'annotation'] = '["SCRACHY THROAT","RUNNY NOSE"]'
df.loc[13238, 'location'] = '["293 307","321 331"]'

df.loc[13297, 'annotation'] = '["without improvement when taking tylenol","without improvement when taking ibuprofen"]'
df.loc[13297, 'location'] = '["182 221","182 213;225 234"]'

df.loc[13299, 'annotation'] = '["yesterday","yesterday"]'
df.loc[13299, 'location'] = '["79 88","409 418"]'

df.loc[13845, 'annotation'] = '["headache global","headache throughout her head"]'
df.loc[13845, 'location'] = '["86 94;230 236","86 94;237 256"]'

df.loc[14083, 'annotation'] = '["headache generalized in her head"]'
df.loc[14083, 'location'] = '["56 64;156 179"]'

In [8]:
df.annotation = df.annotation.apply(lambda x: ast.literal_eval(x))
df.location = df.location.apply(lambda x: ast.literal_eval(x))

In [9]:
# NEW on TPU in TensorFlow 24: shorter cross-compatible TPU/GPU/multi-GPU/cluster-GPU detection code

try: # detect TPUs
    tpu  = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu )
    tf.tpu.experimental.initialize_tpu_system(tpu )
    strategy = tf.distribute.TPUStrategy(tpu )
    print('Using TPU')
except ValueError: # detect GPUs
    tpu = None
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)


AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of accelerators:  1
REPLICAS: 1


In [10]:
MODEL_NAME = 'bert-base-uncased'

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,normalization=True)

In [12]:
def getExpanded(x):
    ex_list = []
    for val in x:
        lst = val.split(";")
        for lst1 in lst:
            lst2 = lst1.split(" ")
            ex_list.extend(list(range(int(lst2[0]), int(lst2[1]))))
    return ex_list

In [13]:
SEQUENCE_LENGTH = 512
def buildTrainingData(df):
  input_ids = []
  attention_mask = []
  offset_mapping = []
  answer = []
  
  for i in tqdm(range(len(df))):
    x = tokenizer.encode_plus(df.loc[i].pn_history, max_length=SEQUENCE_LENGTH, padding='max_length',truncation=True, return_offsets_mapping=True)
    input_ids.append(x['input_ids'])
    attention_mask.append(x['attention_mask'])
    offset_mapping.append(x['offset_mapping'])
    answer.append([1 if x['offset_mapping'][j][0] in getExpanded(df.location.values[i]) else 0 for j in range(SEQUENCE_LENGTH)])
    
  return np.vstack(input_ids), np.vstack(attention_mask), offset_mapping, np.vstack(answer)

def buildTestData(df):
  input_ids = []
  attention_mask = []
  offset_mapping = []
  
  for i in tqdm(range(len(df))):
    x = tokenizer.encode_plus(df.loc[i].pn_history, max_length=SEQUENCE_LENGTH, padding='max_length',truncation=True, return_offsets_mapping=True)
    input_ids.append(x['input_ids'])
    attention_mask.append(x['attention_mask'])
    offset_mapping.append(x['offset_mapping'])
  return np.vstack(input_ids), np.vstack(attention_mask), offset_mapping

In [14]:
data = buildTrainingData(df)

100%|██████████| 14300/14300 [02:13<00:00, 106.91it/s]


In [15]:
bert_model = TFAutoModel.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [16]:
input_ids = tf.keras.layers.Input(shape = (SEQUENCE_LENGTH,), name = 'input_ids', dtype = tf.int32)
attention_mask = tf.keras.layers.Input(shape = (SEQUENCE_LENGTH,), name = 'attention_mask', dtype = tf.int32)

In [17]:
bert_layer = bert_model(input_ids=input_ids,attention_mask=attention_mask)[0]

In [18]:
#out = tf.keras.layers.Dense(20,activation='relu')(debert_layer)
out = tf.keras.layers.Dropout(0.2)(bert_layer)
out = tf.keras.layers.Dense(1, activation='sigmoid')(out)

In [19]:
model = tf.keras.Model(inputs = [input_ids, attention_mask], outputs=out)
model.layers[2].trainable = False

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [21]:
loss = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(0.01)

In [22]:
model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

In [25]:
hostory = model.fit(x = [data[0],data[1]], y = data[3], batch_size=5)

  42/2860 [..............................] - ETA: 57:15 - loss: 3.8969 - acc: 0.7052

KeyboardInterrupt: ignored